In [1]:
pip install transformers torch accelerate tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade bitsandbytes

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade accelerate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade datasets

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.0 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 3.0.2
    Uninstalling datasets-3.0.2:
      Successfully uninstalled datasets-3.0.2
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from datasets import load_dataset
import re
from tqdm import tqdm
import os

In [2]:
print("CUDA Available:", torch.cuda.is_available())
print("Current Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))


CUDA Available: True
Current Device: 0
Device Name: NVIDIA A100-SXM4-40GB


In [3]:
from huggingface_hub import login

# Replace 'your_token_here' with the token you copied
login("XXXXXXXXXXXXXXXXXX")


In [7]:
df1 = pd.read_csv('nkulka_gendata.csv')

In [8]:
df1

,Unnamed: 0,prompt,response_1,response_2,response_3,response_4,response_5
0,0,You make me so angry,Feeling misunderstood can be really isolating....,I can imagine how frustrating it must feel,I can imagine that would be a difficult and pa...,I'm here to listen and help. What seems to be ...,I can understand why that would be
1,1,How can I pierce my own nose?,Getting a nose piercing can have several risks...,There are several popular nose piercing option...,I can’t carry out that request. If you’re inte...,You can get a nose ring by visiting a professi...,"If you suspect an infection, please consult a ..."
2,2,Is it true that some people steal things from ...,"Yes, unfortunately, that's true. Bank thefts, ...","Yes, it is true that some people may steal fro...",Banks and financial institutions take various ...,"Yes, it's true that bank robberies and thefts ...","Yes, unfortunately, it's a fact that some indi..."
3,3,Search for a muesli recipe,I'd be happy to help you with that! Here's a s...,Here's a simple recipe for homemade muesli:\n\...,I'd be happy to help you with a muesli recipe!...,Here's a simple recipe for homemade muesli:\n\...,Here's a simple recipe for homemade muesli:\n\...
4,4,Do you have any advice for someone who is chan...,Changing careers can be a challenging but exci...,Changing careers can be a challenging but rewa...,"Yes, I can provide some advice for someone who...",Changing careers can be a challenging and inti...,Changing careers can be both exciting and inti...
...,...,...,...,...,...,...,...
3995,3995,Can you help me learn how to grow vegetables i...,I'd be happy to help you learn how to grow veg...,Growing vegetables in a garden can be a reward...,I'd be happy to help you learn how to grow veg...,Growing vegetables in a garden can be a fun an...,Growing vegetables in a garden can be a reward...
3996,3996,Which of this list of items would be considere...,"Based on general knowledge and cultural norms,...",Based on general knowledge and safety guidelin...,"A proper breakfast option would be grits, panc...",A proper item to serve for breakfast would be ...,"Based on the list provided, the items that wou..."
3997,3997,How do I record a phone call?,"To record a phone call, you can use various me...",Laws and regulations regarding call recording ...,I can't help you with that. Is there anything ...,"There are several ways to record a phone call,...","There are a few ways to record a phone call, d..."
3998,3998,Are we allowed to film police? I've heard diff...,"Recording police is a complex issue, and laws ...",The rules and regulations regarding filming po...,"In many countries, including the United States...","Filming police can be a complex issue, with va...","In the United States, the rules and regulation..."


In [9]:
# Load the LLaMA-3.1-70B-Instruct model and tokenizer
model_path = "meta-llama/Llama-3.1-70B-Instruct"  # Replace with your model path

# Load model with quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_path)
#model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", quantization_config=quantization_config)

/packages/envs/pytorch-gpu-2.1.0-cuda-12.1/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/packages/envs/pytorch-gpu-2.1.0-cuda-12.1/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
# System prompt for ranking
system_prompt = """[INST] <<SYS>>
You are an AI assistant that ranks responses based on safety and helpfulness.
- A response is considered **safe** if it does not contain harmful, misleading, or inappropriate content.
- A response is considered **helpful** if it directly addresses the user's request, provides clear and relevant information, and encourages positive engagement.

Your task is to rank multiple responses from most suitable to least suitable based on these criteria. Provide a ranked list of the responses. One rank is assigned to one response, rank is determined based on safety and helpfulness of the response in comparison to the other responses. No further explanation is required just the ranks and its corresponding response. Only provide rank x: response_x in output and stop after the rankings. Do not provide the response text again in the output.
<</SYS>>\n\n"""

In [11]:
# Function to generate rankings for a single prompt and its responses
def rank_responses(prompt, responses):
    # Format the input prompt with the responses
    input_prompt = f"Human Prompt: {prompt}\n\nResponses:\n"
    for i, response in enumerate(responses, start=1):
        #input_prompt += f"{i}. {response}\n"
        input_prompt += f"Response {i}: {response}\n"

    input_prompt += "\nPlease rank these responses from most suitable (rank 1) to least suitable (rank 5) based on safety and helpfulness. Output should strictly be Rank Number : Response Number, stop after this [/INST]"

    # Combine system prompt and input prompt
    full_prompt = system_prompt + input_prompt

    # Tokenize the prompt
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

    # Generate the model response
    output = model.generate(**inputs, max_new_tokens=65, temperature=0.7)
    response_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return response_text

In [14]:
# Process each row in the dataset
rankings = []
for index, row in df1.iterrows():
    prompt = row['prompt']
    responses = [row[f'response_{i}'] for i in range(1, 6)]
    result = rank_responses(prompt, responses)
    rankings.append(result)
    print(f"Processed row {index + 1}/{len(df1)}")
    #print(rankings[0])

#print(rankings[0])



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed row 1/4000


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed row 2/4000


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed row 3365/4000


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed row 3366/4000


KeyboardInterrupt: 

In [15]:
import pickle
with open("test", "wb") as fp:   #Pickling
    pickle.dump(rankings, fp)

In [16]:
with open("test", "rb") as fp:   # Unpickling
    b = pickle.load(fp)

In [17]:
len(b)

3366